In [1]:
import numpy as np
import re
import nltk
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch_geometric.data import HeteroData
import torch
import torch_geometric.transforms as T
from sklearn.preprocessing import RobustScaler
import pickle

2024-07-09 18:41:53.642812: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-09 18:41:54.557076: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-09 18:41:54.916084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 18:41:55.532807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 18:41:55.536346: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 18:41:56.507282: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
# Read CSV file
file_path = r"/workspaces/rumour-detection-pheme/Dataset creation/charliehebdo-all-rnr-threads.csv"
df = pd.read_csv(file_path)


In [3]:
df.groupby('rumour')['id'].nunique()

rumour
0    1555
1     447
Name: id, dtype: int64

In [4]:
df.time = pd.to_datetime(df.time, format='%a %b %d %H:%M:%S +0000 %Y')
df.reply_time = pd.to_datetime(df.reply_time, format='%a %b %d %H:%M:%S +0000 %Y')

### Time for replies and Number of replies

In [5]:
 df['time_diff']=(df.reply_time - df.time).dt.total_seconds()/60

In [6]:
df['reply_number'] = df.groupby('id')['time_diff'].rank(method='dense')

#### Number of replies x Retweet counts

In [7]:
df_posts = df[['id','text','followers','favorite_count','retweet_count','verified',\
  'rumour','user_id']].drop_duplicates().merge(df.groupby(['id']).agg(replies=(\
  'time_diff','count'),first_time_diff=('time_diff','first')).reset_index(),\
  on="id",how="left")

#### Word embeddings

In [8]:
# Download NLTK stopwords
nltk.download('stopwords')

english_stopwords = nltk.corpus.stopwords.words('english')

# function for cleaning data
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt


def clean_text(
    string: str,
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=english_stopwords) -> str:
    """
    A method to clean text
    """
    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # Removing the punctuations
    for x in string.lower():
        if x in punctuations:
            string = string.replace(x, "")

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df_posts['clean_text'] = np.vectorize(remove_pattern)(df_posts['text'], "@[\w]*")
df_posts['clean_text'] = df_posts['clean_text'].str.replace("[^a-zA-Z#]", " ").apply(clean_text)

In [10]:
# create the dictionary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_posts['clean_text'])

In [11]:
def embedding_vocab(filepath, word_index,embedding_dim):
    vocab_size = len(word_index) + 1


    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix_vocab

In [12]:
#!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [13]:
embedding_dim = 100
embedding_matrix_vocab = embedding_vocab(
    'glove.6B.100d.txt', tokenizer.word_index,
embedding_dim)

In [14]:
# Tokenize text
sequences = tokenizer.texts_to_sequences(df_posts['clean_text'])

# Padding sequences if necessary
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Function to get embeddings for a sequence
def get_embeddings(sequence, embedding_matrix):
    embeddings = []
    for idx in sequence:
        embeddings.append(embedding_matrix[idx])
    return np.array(embeddings)

# Apply function to get embeddings for each sequence
df_posts['embeddings'] = [get_embeddings(seq, embedding_matrix_vocab) for seq in sequences]

In [15]:
array_avg = []
for i in df_posts.embeddings:
  array_avg.append(np.mean(i,axis=0))
df_posts['embeddings_avg'] = array_avg

#### Reply embeddings

In [16]:
df['clean_reply_text'] = np.vectorize(remove_pattern)(df['reply_text'], "@[\w]*")
df['clean_reply_text'] = df['clean_reply_text'].str.replace("[^a-zA-Z#]", " ").apply(clean_text)

In [17]:
# create the dictionary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_reply_text'])

In [18]:
embedding_dim = 100
embedding_matrix_vocab = embedding_vocab(
    'glove.6B.100d.txt', tokenizer.word_index,
embedding_dim)

In [19]:
# Tokenize text
sequences = tokenizer.texts_to_sequences(df['clean_reply_text'])

# Padding sequences if necessary
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Apply function to get embeddings for each sequence
df['reply_embeddings'] = [get_embeddings(seq, embedding_matrix_vocab) for seq in sequences]

In [20]:
array_avg = []
for i in df.reply_embeddings:
  array_avg.append(np.mean(i,axis=0))
df['reply_embeddings_avg'] = array_avg

/home/codespace/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/codespace/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [21]:

df['reply_embeddings_avg'] = df['reply_embeddings_avg'].apply(lambda x: x if isinstance(x, np.ndarray) else np.zeros(100))

In [22]:
df = df.drop(columns=['Unnamed: 0'])

#### Saving cleaned csv's

In [23]:
df.to_pickle('replies_charlie_hebdo.pkl')

In [46]:
df_posts.to_pickle("posts_charlie_hebdo.pkl")

#### Creat Graph

In [25]:
df_replies=df

In [26]:
#df_replies = pd.read_csv("replies_charlie_hebdo.csv")
#df_posts = pd.read_csv("posts_charlie_hebdo.csv")
#df_replies = df_replies[df_replies.reply_verified.isnull()==False]

In [27]:
#df_post_not_rumour = df_posts[df_posts.rumour==0]
#total_samples = int(df_posts[df_posts.rumour==1].count()[0]*1.1)

# Get the proportion of each value in the 'replies' column
#value_counts = df_post_not_rumour['replies'].value_counts(normalize=True)

# Calculate the number of samples for each value
#samples_per_value = (value_counts * total_samples).round().astype(int)

# Sample the required number of rows for each value in 'replies'
#sampled_dfs = []
#for value, num_samples in samples_per_value.items():
#    sampled_dfs.append(df_post_not_rumour[df_post_not_rumour['replies'] \
#                       == value].sample(num_samples, random_state=42))

# Concatenate all the samples into a single dataframe
#sampled_df = pd.concat(sampled_dfs)

# Shuffle the resulting dataframe
#sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
#df_posts = pd.concat([sampled_df,df_posts[df_posts.rumour==1]])\
#             .sample(frac=1,random_state=42).reset_index(drop=True)

#df_replies = df_replies[df_replies.id.isin(df_posts.id.values)][['id','reply_id',\
#             'reply_followers','reply_user_id','reply_verified',"rumour",\
#              "user_id","reply_embeddings_avg","time_diff"]]

#### Torch Data

In [28]:
# One-hot encoding
df_posts['verified'] = df_posts['verified'].astype('str').str.\
             replace(' ', '').replace('True', '1').replace('False', '0')\
             .astype('int64')

df_posts = pd.concat([df_posts, pd.get_dummies(\
                          df_posts["verified"],dtype=int)], axis=1, join='inner')
df_posts.drop(["verified"], axis=1, inplace=True)
df_posts.rename(columns={1:'verified',0:'no_verified'},inplace=True)

In [29]:
# One-hot encoding
df_replies['reply_verified'] = df_replies['reply_verified'].astype('str').str.\
             replace(' ', '').replace('True', '1').replace('False', '0')\
             .astype('int64')

df_replies = pd.concat([df_replies, pd.get_dummies(\
                          df_replies["reply_verified"],dtype=int)], axis=1, join='inner')
df_replies.drop(["reply_verified"], axis=1, inplace=True)
df_replies.rename(columns={1:'reply_verified',0:'reply_no_verified'},inplace=True)

In [30]:

# Get unique values from the column
unique_values = df_posts['user_id'].unique()

# Create a dictionary to map values to IDs
value_to_id = {value: i for i, value in enumerate(unique_values)}

# Map the values to IDs using the dictionary
#retweets_node_features['index'] = retweets_node_features['reply_user_id'].map(value_to_id)
post_map  = value_to_id

#Only keep features
post_features = df_posts[["followers", "favorite_count","retweet_count","no_verified","verified",\
                          "first_time_diff"]]


In [31]:
embeddings_avg = []
for embeddings in df_posts['embeddings_avg']:
  #embeddings= i.replace('[','').replace(']','').replace('\n','').split(' ')
  #embeddings =  [float(item) for item in embeddings if item != '']
  embeddings_avg.append(embeddings)

In [159]:


# Initialize the Robust Scaler
scaler = RobustScaler()

# Assuming data is a DataFrame containing your dataset
scaled_features = scaler.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])

# Convert the scaled features back to a DataFrame
scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])

# Add the binary features back to the scaled data
scaled_data['no_verified'] = post_features['no_verified']
scaled_data['verified'] = post_features['verified']
post_features = scaled_data

In [32]:
# Convert to numpy
x1 = np.concatenate((post_features.to_numpy(),np.array(embeddings_avg)),axis=1)
#x1 = post_features.to_numpy()
x1.shape # [num_movie_nodes x movie_node_feature_dim]

(2002, 106)

In [33]:
# Select node features
retweets_node_features = df_replies[["reply_followers", "reply_no_verified", "reply_verified","reply_user_id","time_diff"]]

# Get unique values from the column
unique_values = retweets_node_features['reply_user_id'].unique()

# Create a dictionary to map values to IDs
value_to_id = {value: i for i, value in enumerate(unique_values)}

# Map the values to IDs using the dictionary
#retweets_node_features['index'] = retweets_node_features['reply_user_id'].map(value_to_id)
retweets_id_mapping  = value_to_id




#Only keep features
retweets_node_features = retweets_node_features[["reply_followers", "reply_no_verified", "reply_verified","time_diff"]]


In [34]:
embeddings_avg = []
for embeddings in df_replies['reply_embeddings_avg']:
  #embeddings= i.replace('[','').replace(']','').replace('\n','').split(' ')
  #embeddings =  [float(item) for item in embeddings if item != '']
  embeddings_avg.append(embeddings)

In [163]:
# Initialize the Robust Scaler
scaler = RobustScaler()

# Assuming data is a DataFrame containing your dataset
scaled_features = scaler.fit_transform(retweets_node_features[['reply_followers','time_diff']])

# Convert the scaled features back to a DataFrame
scaled_data = pd.DataFrame(scaled_features, columns=['reply_followers','time_diff'])

# Add the binary features back to the scaled data
scaled_data['reply_no_verified'] = retweets_node_features['reply_no_verified']
scaled_data['reply_verified'] = retweets_node_features['reply_verified']
retweets_node_features = scaled_data


In [35]:
# Convert to numpy
#x2 = np.concatenate((retweets_node_features.to_numpy(),np.array(embeddings_avg)),axis=1)
x2 = retweets_node_features.to_numpy()
x2.shape # [num_movie_nodes x movie_node_feature_dim]

(19050, 4)

In [36]:
# Extract labels
labels = df_posts.rumour
y = labels.to_numpy()
y.shape

(2002,)

In [37]:
# Map post IDs
#post_map = posts_id_mapping.reset_index().set_index("id").to_dict()
df_replies["user_id"] = df_replies['user_id'].map(post_map).astype(int)
# Map user IDs
#retweets_map = retweets_id_mapping #retweets_id_mapping.reset_index().set_index("reply_user_id").to_dict()
df_replies["reply_id"] = df_replies["reply_user_id"].map(retweets_id_mapping)#.astype(int)

In [38]:
edge_index = df_replies[["user_id", "reply_id"]].values.transpose()
edge_index # [2 x num_edges]

array([[    0,     0,     0, ...,    25,    25,    25],
       [    0,     1,     2, ...,   284,  1810, 14464]])

In [39]:

# Number of rows
num_rows = x1.shape[0]

# Desired proportions
train_proportion = 0.70
val_proportion = 0.15
test_proportion = 0.15

# Generate a list of indices and shuffle them
indices = np.arange(num_rows)
np.random.shuffle(indices)

# Calculate the split indices
train_end = int(train_proportion * num_rows)
val_end = train_end + int(val_proportion * num_rows)

# Split the indices into train, validation, and test
train_indices = indices[:train_end]
val_indices = indices[train_end:val_end]
test_indices = indices[val_end:]

# Create masks
train_mask = np.zeros(num_rows, dtype=bool)
val_mask = np.zeros(num_rows, dtype=bool)
test_mask = np.zeros(num_rows, dtype=bool)

# Assign True to the corresponding indices
train_mask[train_indices] = True
val_mask[val_indices] = True
test_mask[test_indices] = True


In [40]:
# Initialize the HeteroData object
data = HeteroData()

# Add node features and labels for the 'paper' node type
data['user_id'].x = torch.tensor(x1,dtype=torch.float32)  # Node features dtype=torch.float32
data['user_id'].y = torch.from_numpy(y)  # Node labels (for example, binary classification)
data['user_id'].train_mask = torch.tensor(train_mask)  # Training mask
data['user_id'].val_mask =torch.tensor(val_mask)  # Validation mask
data['user_id'].test_mask = torch.tensor(test_mask)  # Test mask

# Add node features for the 'author' node type
data['reply_id'].x = torch.tensor(x2,dtype=torch.float32) #torch.float32

# Add edges for the (id, retweet, reply_D) relation
data['user_id', 'retweet', 'reply_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x2)))
data = T.ToUndirected()(data)
print(data)


HeteroData(
  user_id={
    x=[2002, 106],
    y=[2002],
    train_mask=[2002],
    val_mask=[2002],
    test_mask=[2002],
  },
  reply_id={ x=[19050, 4] },
  (user_id, retweet, reply_id)={ edge_index=[2, 19050] },
  (reply_id, rev_retweet, user_id)={ edge_index=[2, 19050] }
)


In [41]:
print("train:",data['user_id'].y[train_mask].sum(),data['user_id'].y[train_mask].shape)

train: tensor(306) torch.Size([1401])


In [42]:
print("val:", data['user_id'].y[val_mask].sum(),data['user_id'].y[val_mask].shape)

val: tensor(66) torch.Size([300])


In [43]:
print("test:", data['user_id'].y[test_mask].sum(),data['user_id'].y[test_mask].shape)

test: tensor(75) torch.Size([301])


In [44]:
with open('charlie_hebdo_graph_dataset_node_embeddings.pkl', 'wb') as f:
    pickle.dump(data, f)